<h1><center>Inteligencia Artificial</center></h1>


<h2><center>Tarea 5 - Inferencia Bayesiana y EDA (Exploración de Datos)</center></h2>

---

#### El **objetivo** de esta tarea es realizar un análisis exploratorio básico sobre el conjunto de datos proporcionado (pedidos de clientes a lo largo del tiempo). 


* Además, deberán agregar su informacción correspondiente del equipo que hayan formado, así como el link de github correspondiente al cual cargarán la notebook (ejecutada previamente), para visualizar las respuestas.

---
#Aquí sus datos

Daniel Zubieta Ambriz A01651206 https://github.com/Dazuam/SistemasInteligentes

Carlos Aguirre Orozco A01336319 https://github.com/Lucer9/Sistemas-

Axel Octavio Meinguer Estrada A01653448 https://github.com/alupusmc/SistemasInteligentes

Alister Estrada Cueto A01336767 https://github.com/AlisterEstrada/SistemasInteligente

---

### Objetivo

Predecir qué productos estarán en el próximo pedido de un usuario. El conjunto de datos es anónimo y contiene una muestra de más de 3 millones de pedidos de comestibles de más de 200,000 usuarios de Instacart.

Para cada usuario se entregan 4 y 100 de sus pedidos, con la secuencia de productos comprados en cada pedido

<img src="https://shukka.com/img/xeda.PNG.pagespeed.ic.MVCnc5-70D.png" />

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
from subprocess import check_output
print(check_output(["ls", "../Tarea 5/"]).decode("utf8"))

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
Tarea5_WoAns.ipynb



In [4]:
# <Aqui cargar los otros 6 archivos>
aisles_df = pd.read_csv("../Tarea 5/aisles.csv")
departments_df = pd.read_csv("../Tarea 5/departments.csv")

##lo comente porque genera mucho lag en jupyterlab online, pero funcionan!!
#order_products_prior_df = pd.read_csv("../Tarea 5/order_products__prior.csv")

order_products_train_df = pd.read_csv("../Tarea 5/order_products__train.csv")
orders_df = pd.read_csv("../Tarea 5/orders.csv")
products_df = pd.read_csv("../Tarea 5/products.csv")

In [5]:
#Así se despliega información del cabezal del dataframe
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
#Mostrar header de prior y train
order_products_train_df.head()

In [ ]:
#Aqui su código
order_products__prior_df.head()

## ¿Qué contiene cada archivo?



# Aquí su respuesta

aisles.csv describe productos por pasillo, departments.csv describe departamentos ó areas, order_prdoucts_prior.csv relacion de productos con ordenes de compras, order_prdoucts_train.csv relacion d eprdouctos con departamenteos y sus pasillos. order.csv tiene info de pediddos y los datos de compra, usuarios, etc. products.csv da informacion de los productos

-----

Como se mencionó anteriormente, en este conjunto de datos, se dan de 4 a 100 pedidos de un cliente (lo veremos más adelante) y necesitamos predecir los productos que se volverán a ordenar. Así que el último pedido del usuario se ha extraído y dividido en conjuntos de prueba y de entrenamiento. 

Todas las informaciones de pedidos anteriores del cliente están presentes en el archivo order_products_prior. También podemos notar que hay una columna en el archivo orders.csv llamada eval_set que nos dice a cuál de los tres conjuntos de datos (previo, entrenamiento o prueba) va la fila dada.

-----

Primero obtengamos el recuento de filas en cada uno de los tres conjuntos.

In [ ]:
cnt_srs = orders_df.eval_set.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Eval set type', fontsize=12)
plt.title('Count of rows in each dataset', fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
def get_unique_count(x):
    return len(np.unique(x))

cnt_srs = orders_df.groupby("eval_set")["user_id"].aggregate(get_unique_count)
cnt_srs

# ¿Qué significan los valores anterior? Suponiendo que teníamos un total de clientes...

### Aquí su respuesta

(no se pudo correr de forma estable order_products__prior.csv porque es demasiado pesado para jupyterlab online)
Hay 206209 clientes en las muestras en total, 131209 son para usar en el train
y se lograron predecir 7500

-----

Ahora validemos la afirmación de que se dan de 4 a 100 pedidos de un cliente.

In [ ]:
cnt_srs = orders_df.groupby("user_id")["order_number"].aggregate(np.max).reset_index()
cnt_srs = cnt_srs.order_number.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[2])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Maximum order number', fontsize=12)
plt.xticks(rotation='vertical')
#Aquí su código para mostrar gráfico

Por lo tanto, no hay pedidos de menos de 4 y tiene un límite máximo de 100 como se indica en la página de datos.

Ahora veamos cómo cambia el hábito de hacer pedidos con el día de la semana.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="order_dow", data=orders_df, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of order by week day", fontsize=15)
#Aquí su código para mostrar gráfico

## ¿Cuáles són los días con mayor y menor cantidad de pedidos?

## Aquí su respuesta

El dia de la semana 0 y 1 (fines de semana) son los dias de mas pedidos, el dia 4 es el de menos pedidos (semana)

Ahora veremos cómo es la distribución con respecto a la hora del día.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="order_hour_of_day", data=orders_df, color=color[1])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)
#Aquí su código para mostrar gráfico

# ¿En qué periodo de tiempo se realiza la mayor cantidad de pedidos? 

Mientras es de día

Ahora combinemos el día de la semana y la hora del día para ver la distribución.

In [ ]:
grouped_df = orders_df.groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'order_number')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df)
plt.title("Frequency of Day of week Vs Hour of day")
plt.show()

# ¿Cuándo es el mejor momento para realizar pedidos acorde a lo anterior?

---

## Aquí su respuesta

Fines de semana en dia 0 de noche y dias 1 en la mañana

Ahora revisemos el intervalo de tiempo entre los pedidos.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="days_since_prior_order", data=orders_df, color=color[3])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Days since prior order', fontsize=12)
#Aquí su código para mostrar gráfico

# Parece que los clientes hacen pedidos... ¿cada cuánto?

### Aquí su respuesta

Hay un pico en el dia 7 del mes siendo inicios de mes y claramente hay un pico de compras en el ultimo dia del mes, quiza hay alguna relacion ofertas de fin d emes

---

# Según la tendencia, ¿hay otros días que muestren picos pequeños?

### Aquí su respuesta

Hay picos cada 15 dias, una tendencia a los cortes y quincenas (pagos)

---


Dado que nuestro objetivo es averiguar los reordenamientos, veamos el porcentaje de reorden en la serie anterior y la serie de trenes.

In [ ]:
# percentage of re-orders in prior set #
order_products_prior_df.reordered.sum() / order_products_prior_df.shape[0]

In [ ]:
# percentage of re-orders in train set #
order_products_train_df.reordered.sum() / order_products_train_df.shape[0]

## ¿Qué significan los valores anteriores?

### Aquí su respuesta

59% de probabilidad de que se se vuleva a ordernar ese producto

---

### No hay productos reordenados:

Ahora que hemos visto que el 59% de los productos se vuelven a pedir, también habrá situaciones en las que no se vuelva a pedir ninguno de los productos. Comprobemos eso ahora.

In [ ]:
grouped_df = order_products_prior_df.groupby("order_id")["reordered"].aggregate("sum").reset_index()
grouped_df["reordered"].loc[grouped_df["reordered"]>1] = 1
grouped_df.reordered.value_counts() / grouped_df.shape[0]

In [ ]:
grouped_df = order_products_train_df.groupby("order_id")["reordered"].aggregate("sum").reset_index()
grouped_df["reordered"].loc[grouped_df["reordered"]>1] = 1
grouped_df.reordered.value_counts() / grouped_df.shape[0]

# ¿Y en esta sección?

### Aquí su respuesta

6% de los valores de entrenamiento no tiene productos reondenados, probablemnete al prior ser el doble, haya un 12% de no reordenados en prior

----

Ahora veamos la cantidad de productos comprados en cada pedido.

In [ ]:
grouped_df = order_products_train_df.groupby("order_id")["add_to_cart_order"].aggregate("max").reset_index()
cnt_srs = grouped_df.add_to_cart_order.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Number of products in the given order', fontsize=12)
#Aquí el código faltante para mostrar gráfico

# ¿Cuál es el valor más alto?

### Aquí su respuesta

Maximo valor es 5

---

Antes de explorar los detalles del producto, veamos también los otros tres archivos.

In [ ]:
products_df.head()

In [ ]:
aisles_df.head()

In [ ]:
departments_df.head()

Ahora fusionemos estos detalles del producto con los detalles de order_prior.

In [ ]:
order_products_prior_df = pd.merge(order_products_prior_df, products_df, on='product_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, aisles_df, on='aisle_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, departments_df, on='department_id', how='left')
order_products_prior_df.head()

In [ ]:
cnt_srs = order_products_prior_df['product_name'].value_counts().reset_index().head(20)
cnt_srs.columns = ['product_name', 'frequency_count']
cnt_srs

# ¿Qué podemos inferir de los resultados anteriores?


### aquí su respuesta

Hay una predominancia en prdocutos organicos, comida y frutas

---

Ahora echemos un vistazo a los pasillos importantes.

In [ ]:
cnt_srs = order_products_prior_df['aisle'].value_counts().head(20)
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[5])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Aisle', fontsize=12)
#Aquí sus códigos...

# ¿Cuáles son los pasillos con mayor valor?

### Aquí su respuesta

Frutas y verduras

----

### Distribución departamental:

Veamos ahora la distribución inteligente del departamento.


In [ ]:
plt.figure(figsize=(10,10))
temp_series = order_products_prior_df['department'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))
plt.pie(sizes, labels=labels, 
        autopct='%1.1f%%', startangle=200)
plt.title("Departments distribution", fontsize=15)

# ¿Cuál es el departamento más grande? y ¿por cuales es seguido?

### Aquí su respuesta

Produce es el departamento mayor, se le sigue dairy eggs, snacks y luego beverages

---

### Relación de pedidos por departamento:

In [ ]:
grouped_df = order_products_prior_df.groupby(["department"])["reordered"].aggregate("mean").reset_index()
grouped_df2 = grouped_df.copy()
plt.figure(figsize=(12,8))
sns.pointplot(grouped_df['department'].values, grouped_df['reordered'].values, alpha=0.8, color=color[2])
#Aquí falta algo...

# ¿Cuáles son los departamentos con mayor y menor cantidad de proporción de pedidos?

### Aquí su respuesta

Mayor dairy eggs; menor personal care

---


### Pasillo - Relación de pedidos:

In [ ]:
grouped_df = order_products_prior_df.groupby(["department_id", "aisle"])["reordered"].aggregate("mean").reset_index()

fig, ax = plt.subplots(figsize=(12,20))
ax.scatter(grouped_df.reordered.values, grouped_df.department_id.values)
for i, txt in enumerate(grouped_df.aisle.values):
    ax.annotate(txt, (grouped_df.reordered.values[i], grouped_df.department_id.values[i]), rotation=45, ha='center', va='center', color='green')
#Aqui falta algo...

### Agregar al carrito - Relación de pedidos:

Exploremos ahora la relación entre cómo el orden de agregar el producto al carrito afecta la tasa de reorden.

In [ ]:
order_products_prior_df["add_to_cart_order_mod"] = order_products_prior_df["add_to_cart_order"].copy()
order_products_prior_df["add_to_cart_order_mod"].loc[order_products_prior_df["add_to_cart_order_mod"]>70] = 70
grouped_df = order_products_prior_df.groupby(["add_to_cart_order_mod"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.pointplot(grouped_df['add_to_cart_order_mod'].values, grouped_df['reordered'].values, alpha=0.8, color=color[2])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Add to cart order', fontsize=12)
plt.title("Add to cart order - Reorder ratio", fontsize=15)
plt.xticks(rotation='vertical')
plt.show()

# ¿Qué podemos interpretar de este gráfico? ¿Hay alguna relación a la probabilidad de volver a elegir dicho producto?


### Aquí su respuesta

Es posible que los productos que primero se agarran son aquellos que probablemente se vuelvan a comprar, poseen una prioridad

---


### Reordenar la proporción por variables basadas en el tiempo:

In [ ]:
order_products_train_df = pd.merge(order_products_train_df, orders_df, on='order_id', how='left')
grouped_df = order_products_train_df.groupby(["order_dow"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.barplot(grouped_df['order_dow'].values, grouped_df['reordered'].values, alpha=0.8, color=color[3])
#Aquí falta algo...

In [ ]:
grouped_df = order_products_train_df.groupby(["order_hour_of_day"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.barplot(grouped_df['order_hour_of_day'].values, grouped_df['reordered'].values, alpha=0.8, color=color[4])
plt.ylabel('Reorder ratio', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)

In [ ]:
grouped_df = order_products_train_df.groupby(["order_dow", "order_hour_of_day"])["reordered"].aggregate("mean").reset_index()
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'reordered')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df)


# ¿Qué podemos inferir de todo lo anterior? ¿Las tasas de reordenar algo en qué momento del día tienen mayor frecuencia?


### Aquí su respuesta

Hay una tendencia en las mañanas para reorden

----


In [ ]:
pip install --upgrade plotly

In [ ]:
## A partir del siguiente árbol de asociación... 
order_products_prior_df = pd.merge(order_products_prior_df, products_df, on='product_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, aisles_df, on='aisle_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, departments_df, on='department_id', how='left')
order_products_prior_df.head()

In [ ]:
test = order_products_prior_df.sample(n = 1500)  

In [ ]:
test

In [ ]:
import plotly.express as px 
    
fig = px.treemap(test, path=['department', 'product_name'], 
                 values='add_to_cart_order') 
  
fig.show()

¿Cuántos productos únicos se ofrecen en cada departamento / pasillo?

El tamaño de las cajas muestra la cantidad de productos en cada categoría.

### Da una breve explicación de los primeros 3 recuadros

Recuadros externos indica las areas con mas productos vendidos (produce, dairy eggs y snacks), dentro de esto se puede saber del area que productos especifcicos mas se venden respectivamente.